# Import Dependecies

In [1]:
import math, pydot, graphviz
import numpy as np
import pandas as pd
from collections import defaultdict
# Split the data into train and test sets
# importing relevant libraries
from sklearn.model_selection import train_test_split
from book_funcs import *

In [2]:
import keras
import pydot
import pydotplus
from pydotplus import graphviz
from keras.utils import plot_model
from keras.utils import model_to_dot
keras.utils.pydot = pydot
import tensorflow as tf
from tensorflow.keras.utils import plot_model
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import FunctionTransformer
from lightfm import LightFM

## Import Data

In [3]:
df = pd.read_csv('/Users/royzalta/Downloads/df_clean_after_pca')

## Data Preprocessing

In [4]:
# delete the rows with rating = 0
df = df[df['rating']!=0]
df_isbn = df['isbn']
num_dict = {}
# Create a dictionary of unique isbn values
for i,j in enumerate(df_isbn.unique()):
    num_dict[j]=i
df['isbn_num'] = df['isbn'].apply(lambda txt: num_dict[txt])
df = df.reset_index(drop = True)

In [6]:
# create a unique books autor id
book_author = df['book_author']
num_dict = {}
for i,j in enumerate(book_author.unique()):
    num_dict[j]=i
df['author_num'] = df['book_author'].apply(lambda txt: num_dict[txt])


## Train Test Split

In [7]:
#split data into train and test
df =reader_encode(df)
train_users, test_users = train_test_split(df, test_size = 0.2 , random_state= 42)
x_train = train_users.drop(columns=['rating'])
y_train = train_users["rating"]

x_test = test_users.drop(columns=['rating'])
y_test = test_users["rating"]

all_users = train_users["user_id"].unique()
all_books = train_users["isbn_num"].unique()
all_authors = train_users["author_num"].unique()


## Implemention of LightFM

In [ ]:
# embedding for user
user_input = tf.keras.layers.Input(shape=(1,), name="user")
user_as_integer = tf.keras.layers.IntegerLookup(vocabulary=all_users)(user_input)
user_embedding = tf.keras.layers.Embedding(input_dim=len(all_users) + 1, output_dim=32)(user_as_integer)
user_bias = tf.keras.layers.Embedding(input_dim=len(all_users) + 1, output_dim=1)(user_as_integer)
# embedding for book
book_input = tf.keras.layers.Input(shape=(1,), name="book")
book_as_integer = tf.keras.layers.IntegerLookup(vocabulary=all_books)(book_input)
book_embedding = tf.keras.layers.Embedding(input_dim=len(all_books) + 1, output_dim=32)(book_as_integer)
book_bias = tf.keras.layers.Embedding(input_dim=len(all_books) + 1, output_dim=1)(book_as_integer)

# embedding for author
author_input = tf.keras.layers.Input(shape=(1,), name="author")
author_as_integer = tf.keras.layers.IntegerLookup(vocabulary=all_authors)(author_input)
author_embedding = tf.keras.layers.Embedding(input_dim=len(all_authors) + 1, output_dim=32)(author_as_integer)

dot_author = tf.keras.layers.Dot(axes=2)([user_embedding, author_embedding])
flatten_author = tf.keras.layers.Flatten()(dot_author)


feature_input = tf.keras.layers.Input(shape=(230,), name="book_data")
feature = tf.keras.layers.Dense(200, activation = 'relu', name="add_some_feature",kernel_regularizer=l2(0.01))(feature_input)
dropout1 = tf.keras.layers.Dropout(0.2, name="drop_some_feature")(feature)
feature2 = tf.keras.layers.Dense(128, activation = 'relu', name="add_some_feature2",kernel_regularizer=l2(0.01))(dropout1)
dropout1_2 = tf.keras.layers.Dropout(0.2, name="drop_some_feature22")(feature2)
feature2_2 = tf.keras.layers.Dense(64, activation = 'relu', name="add_some_feature22",kernel_regularizer=keras.regularizers.l2(0.01))(dropout1_2)

user_feature_input = tf.keras.layers.Input(shape=(176,), name="user_data")
feature_u = tf.keras.layers.Dense(128, activation = 'relu', name="add_some_feature_u",kernel_regularizer=keras.regularizers.l2(0.01))(user_feature_input)
dropout1_u = tf.keras.layers.Dropout(0.3, name="drop_some_feature_u2")(feature_u)
feature2_u = tf.keras.layers.Dense(64, activation = 'relu', name="add_some_feature2_u2",kernel_regularizer=keras.regularizers.l2(0.01))(dropout1_u)


dot = tf.keras.layers.Dot(axes=2)([user_embedding, book_embedding])
add = tf.keras.layers.Add()([dot, user_bias, book_bias])
flatten = tf.keras.layers.Flatten()(add)
con_book_users = tf.keras.layers.concatenate([feature2_2,feature2_u])
feature2_u = tf.keras.layers.Dense(128, activation = 'relu', name="reader_book_type")(con_book_users)

con = tf.keras.layers.concatenate([flatten,flatten_author,feature2_u])#,feature2_u])
dense2 = tf.keras.layers.Dense(128, activation = 'relu', name="dence_layer2",kernel_regularizer=keras.regularizers.l2(0.01))(con)
dropout2 = tf.keras.layers.Dropout(0.2, name="drop_some_feature3")(dense2)
dense3 = tf.keras.layers.Dense(64, activation = 'relu', name="dence_layer3",kernel_regularizer=keras.regularizers.l2(0.01))(dropout2)
dropout3 = tf.keras.layers.Dropout(0.2, name="drop_some_feature4")(dense3)
dense4 = tf.keras.layers.Dense(16, activation = 'relu', name="dence_layer4",kernel_regularizer=keras.regularizers.l2(0.01))(dropout3)
dropout4 = tf.keras.layers.Dropout(0.2, name="drop_some_feature5")(dense4)
dense5 = tf.keras.layers.Dense(8, activation = 'relu', name="dence_layer5",kernel_regularizer=keras.regularizers.l2(0.01))(dropout4)
dropout5 = tf.keras.layers.Dropout(0.2, name="drop_some_feature6")(dense5)

squash = tf.keras.layers.Lambda(lambda x: 9 * tf.nn.sigmoid(x) + 1)(dropout5)



model = LightFM(inputs=[user_input, book_input,author_input,feature_input,user_feature_input], outputs=squash)

# add adam optimizer
model.compile(loss="mse",optimizer = 'adam', metrics=[tf.keras.metrics.MeanAbsoluteError(),tf.keras.metrics.RootMeanSquaredError()])
